In [17]:
# code for Navigation Systems Dijkstra's Algorithm assignment
# by Robin Holfelder and Leon Prucker

# setting the working directory to the current directory
import os
notebook_dir = os.getcwd()
os.chdir(notebook_dir)

# importing necessary libraries
import numpy as np
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from collections import defaultdict

In [16]:
# general data loading
#read nodelist
nodelist = np.loadtxt('nodelist.txt')
nodelist = nodelist.astype(int)
# read arclist
arclist = np.loadtxt('arclist.txt')

#custom cost function
# creating dictionary with keys from 1 to 7
costs = defaultdict(int)
costs[1] = 1000 # assign costs to each clazz key
costs[2] = 1
costs[3] = 10
costs[4] = 1000
costs[5] = 1000
costs[6] = 3
costs[7] = 5

#creating a new column in arclist with custom cost and assigning values to it based on the costs dictionary
arclist = np.hstack((arclist, np.zeros((arclist.shape[0], 1)))) 
for row in arclist:
    row[6] = costs[int(row[5])] * row[2]



arclist = arclist[:, [0,6]] # change this for different cost

# [:, [0, 2]] is distance
# [:, [0, 1]] is time
# [:, [0, 6]] is custom cost (bike riding)


arclist[:, 0] = arclist[:, 0].astype(int)



In [3]:
#source node
vs =  175

# initialize variables
ls = 0 
big_T = {vs} # set of visited nodes
big_P = set() # set of permanent nodes
big_V = list(range(1, len(nodelist))) 
min_distance = float("inf") # initialize minimum distance to infinity

labels = {vs:0} # dictionary to store the distance from source node to each node

In [4]:

n_nodes = len(nodelist)-1 # number of nodes
outgoing = [0] * n_nodes # array with number of nodes as 0s
line_number = 0 
predecessor = {} # dictionary to store the predecessor of each node

neighbor = defaultdict(list) # dictionary with keys as nodes and values as list of tuples with neighbors and cost
for i in range(0, n_nodes):
    outgoing[i] = nodelist[i+1] - nodelist[i]
    for j in range(0, outgoing[i]):
        neighbor[i+1].append(tuple(arclist[line_number+j]))
    line_number = line_number + outgoing[i]




In [5]:
# main dijkstra algorithm

# initialize labels for all nodes to infinity except source node
for vj in big_V:
    if vj == vs:
        labels[vj] = 0
    else:
        labels[vj] = float('inf')

# main loop
while big_T: # while big_T (temporary set) is not empty
    min_distance = float('inf') # initialize minimum distance to infinity
    for vj in big_T: # for each node in big_T
        current_distance = labels[vj] # get the distance from source node to the node
        if current_distance < min_distance: # if the distance is less than minimum distance
            min_distance = current_distance # update minimum distance
            vi = vj # update the node with minimum distance
    big_P.add(vi) # add the node with minimum distance to big_P (permanent set)
    big_T.remove(vi) # remove the node with minimum distance from big_T (temporary set)
    for vj, weight in neighbor[vi]: # for each neighbor of the node with minimum distance
        if vj not in big_P and vj not in big_T: # if the neighbor is not in big_P and not in big_T
            labels[vj] = labels[vi] + weight # update the distance from source node to the neighbor
            predecessor[vj] = vi # update the predecessor of the neighbor
            big_T.add(vj) # add the neighbor to big_T
        elif vj in big_T and labels[vi] + weight < labels[vj]: # if the neighbor is in big_T and the distance from source node to the neighbor is less than the current distance
            labels[vj] = labels[vi] + weight # update the distance from source node to the neighbor
            predecessor[vj] = vi # update the predecessor of the neighbor



In [6]:
# function to get the path from source node to target node
def get_predecessor_path(predecessor, target_node):
    path = []
    current_node = target_node
    while current_node in predecessor: # while the current node has a predecessor
        path.append(current_node) # add the current node to the path
        current_node = predecessor[current_node] # update the current node to its predecessor
    path.append(current_node) 
    path.reverse() # reverse the path to get the path from source node to target node
    return path

# target nodes
target_node = [9328, 6031, 8543]  
paths = {} # dictionary to store the path from source node to each target node

# get the path from source node to each target node
for i in range(0, len(target_node)):
    target_node[i] = int(target_node[i])
    paths[i] = get_predecessor_path(predecessor, target_node[i])


In [7]:
# read in nodepl.txt
nodepl = np.loadtxt('nodepl.txt')
#convert to pandas df
nodepl = pd.DataFrame(nodepl, columns=['x', 'y'])
# add a column with the node number whih is the index
nodepl['node'] = nodepl.index + 1


for key, path in paths.items():
    # turn path into a array with integer values
    path = np.array(path).astype(int)
    # make path into a pandas df with one column
    path = pd.DataFrame(path, columns=['node'])
    # join path with nodepl by node
    path = pd.merge(path, nodepl, on='node', how='left')
    # export path to csv
    path.to_csv(f'path_custom_{key}.csv', index=False) # change file name here for different cost

In [11]:
# function to get starting node
def haversine(lat1, lon1, lat2, lon2): # function to calculate the distance between two coordinates on the Earth (from geomath)
    R = 6371 # radius of the Earth in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon1 - lon2)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Set home coordinates and find nearest node
home_lat, home_lon = 47.065307430925586, 15.44306321533678 # grazbachgasse 34
distances = [haversine(home_lat, home_lon, row['x'], row['y']) for index, row in nodepl.iterrows()]
start_node = np.argmin(distances) + 1 

print(f'Start node: {start_node}')



Start node: 175
